## 1. Keyword search with synonyms

### Synonyms

- doc: https://learn.microsoft.com/en-us/azure/search/search-synonyms?tabs=python%2Crest-assign#create-a-synonym-map
- sample: https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/search/azure-search-documents/samples/sample_synonym_map_operations.py

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import (
    QueryType,
    SearchMode
)

from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import ( 
    SynonymMap, 
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

#print(service_endpoint)
index_name = "korproverb-keywordstd-index"

In [4]:
index_client = SearchIndexClient(service_endpoint, credential=credential)

In [5]:
def create_synonym_map(map_name):
    # [START create_synonym_map]
    synonyms = [
        "노력, 노력하면, 노력과, 노력을",
        "Washington, Wash. => WA",
    ]
    synonym_map = SynonymMap(name=map_name, synonyms=synonyms)

    result = index_client.create_or_update_synonym_map(synonym_map)
    print("Create new Synonym Map 'test-syn-map succeeded")
    # [END create_synonym_map]

def delete_synonym_map(map_name):
    index_client.delete_synonym_map(map_name)

In [6]:
# update a search index
def update_search_index(add_synonym=True):
    if add_synonym:
        meaning_field = SearchableField(name="meaning", type=SearchFieldDataType.String, 
            searchable=True, retrievable=True,
            synonym_map_names=[synonym_map_name])
    else:
        meaning_field = SearchableField(name="meaning", type=SearchFieldDataType.String,
            searchable=True, retrievable=True)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        meaning_field,
        SearchableField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
    ] 

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} updated')

In [7]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [8]:
def azsch_text_query(query, type=QueryType.Simple, mode=SearchMode.Any):

    results = search_client.search(  
        search_text=query,
        query_type=type,
        search_mode=mode,
        search_fields=["meaning"],
        select=["proverb", "meaning", "source", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['type']}")  

In [9]:
azsch_text_query("노력")

Search Results:
1: 6.2826767000: 힘을 넣다 - 어떤 일을 이루려고 시간, 노력, 자금 따위를 투입하다., 관용구
2: 3.9428840000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - 노력은 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, 노력 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담


In [10]:
synonym_map_name = "test-syn-map"
create_synonym_map(synonym_map_name)

update_search_index(add_synonym=True)

Create new Synonym Map 'test-syn-map succeeded
 korproverb-keywordstd-index updated


In [11]:
azsch_text_query("노력")

Search Results:
1: 8.1555120000: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 관용구
2: 6.5316040000: 힘을 돌리다 - 어떤 일을 이루려고 거기에 노력을 들이다., 관용구
3: 6.5175890000: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 속담
4: 6.3806940000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 노력을 들이지 않았는데 저절로 얻어지다., 관용구
5: 6.3524027000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 속담
6: 6.2826767000: 힘을 넣다 - 어떤 일을 이루려고 시간, 노력, 자금 따위를 투입하다., 관용구
7: 6.2167478000: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 속담
8: 5.7782000000: 내 침 발라 꼰 새끼가 제일 - 자기의 노력을 들여 이룩한 성과가 귀중함을 비유적으로 이르는 말., 속담
9: 5.4407950000: 쉬 더운 방이 쉬 식는다 - 힘이나 노력을 적게 들이고 빨리 해 버린 일은 그만큼 결과가 오래가지 못함을 비유적으로 이르는 말., 속담
10: 5.4405427000: 체메 들다 - 남의 사정이나 수단에 의하여 어이없이 돈이나 노력을 대신 부담하다., 관용구


In [12]:
update_search_index(add_synonym=False)

#delete_synonym_map(synonym_map_name)

 korproverb-keywordstd-index updated
